This blog post describes three use cases of the `fumbbl_replays` package.
For the first use case, the functionality of the package is sufficient, with no additional programming needed.
For the other two use cases, additional programming is needed.
In the future, this code could be turned into a function and added to the package.

To install the `fumbbl_replays` python package, follow the instructions on Github.

# Application 1: What's the play



In [ ]:
import sys
from PIL import Image, ImageDraw

with Image.open("doc/Whats the play 04 - Dwarf Vs Elven Union.png") as im:
   # Provide the target width and height of the image
   (width, height) = (im.width // 2, im.height // 2)
   im_resized = im.resize((width, height))
   display(im_resized)

Lets try to reproduce this plot with `fumbbl_replays`.
We start with fetching a Dwarf Roster. 
Then create the positions including playerStates (i.e. the Stunned Troll Slayer). 
Then finally we add the skills.

In [ ]:
import fumbbl_replays as fb

my_roster = fb.fetch_roster("Dwarf")

my_setup = ['setup', ['B3: d14', 'TS4: b15X', 'L10: l17', 
                      'L11: n17', 'L9: j19', 'L5: l20', 'L8: n20', 
                      'L6: j21', 'R1: m21o', 'L7: l22', 'B2: n22']]

positions = fb.create_position(my_roster, my_setup)

fb.add_skill_to_player(positions, "R1", "On the Ball")
fb.add_skill_to_player(positions, "B3", "Guard")
#fb.add_skill_to_player(positions, "TS4", "Block") # TS already comes with block
fb.add_skill_to_player(positions, "TS4", "Mighty Blow")
fb.add_skill_to_player(positions, "L7", "Guard")
fb.add_skill_to_player(positions, "L8", "Guard")
fb.add_skill_to_player(positions, "L9", "Guard")
fb.add_skill_to_player(positions, "L10", "Guard")

Repeat for Elven Union:

In [ ]:
import pandas as pd

my_roster = fb.fetch_roster("Elven Union")

my_setup = ['setup', ['T1: l25', 'C3: i24', 'B6: i23', 
                      'B7: g22', 'L10: i21', 'L11: h20', 'L12: c16', 
                      'L8: c15', 'C2: h11']]

positions2 = fb.create_position(my_roster, my_setup, home_away = 'teamAway')

fb.add_skill_to_player(positions2, "T1", "Accurate")
fb.add_skill_to_player(positions2, "T1", "Leader")
fb.add_skill_to_player(positions2, "C3", "Dodge")
fb.add_skill_to_player(positions2, "C3", "Block")
fb.add_skill_to_player(positions2, "B6", "Dodge")
fb.add_skill_to_player(positions2, "B6", "Strip Ball")
fb.add_skill_to_player(positions2, "B7", "Mighty Blow")
fb.add_skill_to_player(positions2, "L10", "Block")

positions = pd.concat([positions, positions2])

In [ ]:
fb.create_plot(positions, red_team = "teamAway", orientation = 'H', skill_bands = True)

We can reposition the Elven Union players to describe a possible solution to the what's the play puzzle.
This facilitates discussions of alternative plays, and the risks and benefits associated with them.

In [ ]:
positions = fb.move_piece(positions, "teamAway", "T1", "n25")
positions = fb.move_piece(positions, "teamAway", "B6", "n24")
positions = fb.move_piece(positions, "teamAway", "C3", "k25")
positions = fb.move_piece(positions, "teamAway", "B7", "k24")
positions = fb.move_piece(positions, "teamAway", "L10", "h24")
positions = fb.move_piece(positions, "teamAway", "L11", "h25")
positions = fb.move_piece(positions, "teamAway", "L12", "g22")
positions = fb.move_piece(positions, "teamAway", "C2", "g19")

fb.create_plot(positions, red_team = "teamAway", orientation = 'H', skill_bands = True)

The remaining lineman L8 could finally do a dodge and two rushes to end up in g23.

This also immediately gives rise to new feature requests: highlighting squares, drawing arrows, some form of pathfinding (e.g. find shortest path without dice rolling).

# Application 2: Plotting defensive tournament setups

Here the idea is to visualize defensive setups in a tournament setting.

To test the idea, I chose the Tilean Team Cup. It was an online NAF tournament in 2023 on FUMBBL that used the World Cup ruleset.
https://member.thenaf.net/index.php?module=NAF&type=tournaments&func=view&id=7495

NAF tournament director Stimme wrote:

https://www.thenaf.net/2023/05/tournament-director-blog-may-2023/

*Among the individual coaches, Siggi stood out with his Amazons, earning the title of best coach with a flawless record of six wins. Kurjo’s Orcs secured second place, while helborg’s Dark Elves came in third, both with five wins and a draw.* 


In [ ]:
import pandas as pd
import fumbbl_replays as fb

# point this to the location of the CSV datasets
path_to_datasets = '../fumbbl_scraping/datasets/current/'

# FUMBBL matches
target = 'df_matches.csv'
df_matches = pd.read_csv(path_to_datasets + target) 

# # subset on tilean team cup
df_matches = df_matches.query('tournament_id == 59383')

sel1 = df_matches['coach1_ranking'] == 'siggi'
sel2 = df_matches['coach2_ranking'] == 'siggi'

df_matches = df_matches[(sel1 + sel2)]

tilean_replays = df_matches['match_id'].values

tilean_replays = tilean_replays[0:4]

In [ ]:
tilean_replays

In [ ]:
import os

target_dir = 'kickoff_pngs'

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

fullrun = 1

if fullrun:
    id = []
    match_ids = []
    race_defense = []
    race_offense = []

    for match_id in tilean_replays:
        match_id, replay_id, positions, receiving_team, metadata = fb.fetch_data(match_id) 
        plot = fb.write_plot(match_id, positions, receiving_team, metadata, refresh = True, verbose = False)
        id.append(int(replay_id))
        match_ids.append(int(match_id))
        race_defense.append(metadata[4])
        race_offense.append(metadata[5])

    df_replays = pd.DataFrame( {"matchId": match_ids,
                                "replayId": id,
                                "raceOffense": race_offense,
                                "raceDefense": race_defense})
    target = 'kickoff_pngs/df_replays'
    df_replays.to_csv(target + '.csv', index = False)
else:
    # FUMBBL matches
    target = 'kickoff_pngs/df_replays.csv'
    df_replays = pd.read_csv(target)  

In [ ]:
from PIL import Image
Image.open("kickoff_pngs/amazon/1616794_4456080_kickoff_lower_defense.png")

Apparently, coach siggi used an asymmetric setup against his Skaven opponent, with the weakest players on the line as cannon fodder, and positional players with good defensive skills such as block and wrestle further back. The dirty player lineman, Leader Thrower and Guard blocker are safe on row four.

There is also a function that extracts the roster from a replay (N.b. rerolls, apo and inducements not yet implemented):

In [ ]:
my_replay = fb.fetch_replay(match_id = 4447483)
rosters = fb.extract_rosters_from_replay(my_replay)

(rosters
 .query('race == "Amazon"')
 .filter(['short_name', 'positionName', 'playerName', 'skillArrayRoster', 'learned_skills', 'cost', 'recoveringInjury'])
)

# Application 3: Roster development in league play

The last analysis focusses on team development in League play.
I am currently in a league with Gnomes and was curious how to develop the team.
I found a Gnome team with amazing league performance ("We do gnomes") by coach Elyod, one of the top ranking coaches on FUMBBL.
The team has played in "Black Box Trophy 11" (BBT 11) and finished with a W/D/L of 10/2/3. 
What can we learn from how he developed his Gnomes?

For this analysis, the starting point is the `team_id`. This is part of the url of the team on FUMBBL: https://fumbbl.com/p/team?id=1177218
I wrote a function `fetch_team_matches()` to fetch all `match_id`'s for a given team id.
By fetching the replays and extracting the Gnome team roster from each replay, we can track the development.

In [ ]:
import pandas as pd
import fumbbl_replays as fb
from plotnine import *

pd.set_option('display.max_colwidth', None)


## bbt 12 snots
team_id = str(1211200)
# bbt 13 amazons
team_id = str(1211218)
team_id = str(1212484)
team_id = str(1217257)


# bbt 11 gnomes
team_id = str(1177218)

team_matches = fb.fetch_team_matches(int(team_id))

In [ ]:
matches = []
for i in range(len(team_matches)):
    matches.append(team_matches[i]['id'])

matches.sort()

In [ ]:
# fetch rosters from replays
i = 0

for match_id in matches[0:15]:
    print(".", end = '')
    my_replay = fb.fetch_replay(match_id)
    df_positions = fb.extract_rosters_from_replay(my_replay) 
    df_positions = (df_positions
                    .query("teamId == @team_id")
                    .filter(['short_name', 'rosterName' , 'positionName', 'playerName', 'skillArrayRoster', 'learned_skills', 'skill_colors', 'cost', 'recoveringInjury'])
                    .reset_index()
                    )
    df_positions['match_count'] = i+1
    if i == 0:
        res = df_positions
    else:
        res = pd.concat([res, df_positions])
    i = i + 1


In [ ]:
cell_color = []
learned_skill = []

for i in range(len(res)):
    n_skills = len(res.iloc[i]['learned_skills'])
    if n_skills == 0:
        cell_color.append("grey")
        learned_skill.append("-")
    elif n_skills == 1:
        cell_color.append(res.iloc[i]['skill_colors'][n_skills - 1])
        learned_skill.append(res.iloc[i]['learned_skills'][n_skills - 1])
    else:
        cell_color.append("black")
        learned_skill.append(">1")        

res['cell_color'] = cell_color
res['learned_skill'] = learned_skill
res = res.drop(columns = ['short_name'])

In [ ]:
res = res.query('learned_skill != "Loner" & recoveringInjury == "None" ')

In [ ]:
rosterName = res['rosterName'].unique()[0]
roster = fb.fetch_roster(rosterName).filter(['positionName', 'shorthand'])

player_list = (res
    .groupby(['playerName', 'positionName'], as_index = False)
    .agg(count = ('index', 'count'), first_match = ('match_count', 'min'))
    .merge(roster, on = 'positionName')
    .sort_values(['positionName', 'first_match'])
)

player_list['positionNr'] = player_list.groupby(['positionName']).cumcount() + 1
# create shorthand id
player_list['short_name'] = player_list['shorthand'] + player_list['positionNr'].astype(str)

In [ ]:
res = res.merge(player_list.filter(['playerName', 'shorthand', 'short_name', 'positionNr']), on = 'playerName')

In the Black Box Trophy 11, teams start off with 1M gold, and play 15 matches against random opponents in the Blackbox division.
It uses the rulebook team development rules (earning SPP, leveling up players etc).

This is the roster that Elyod ended up with after 15 games.


In [ ]:
(
    ggplot(res, aes(x="factor(match_count)", y="reorder(short_name, cost)"))
    + geom_tile(aes(fill = "cell_color", width=0.95, height=0.95))
    + geom_text(aes(label="learned_skill"), size=6)
    + scale_fill_identity()
    #+ coord_equal(expand=False)  # new
    + theme(figure_size=(12, 6))  # new
    + theme(legend_position='none')
    + labs(x="League game number", y="Players", title= rosterName + " team development BBT")
)

In [ ]:
res.query('match_count == 15')